In [62]:
# from google.colab import drive
# drive.mount('/content/drive')

In [63]:
# cd /content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification

# Imports

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin

from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
    
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import mean_squared_error,accuracy_score,f1_score,roc_auc_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb
# import xgboost as xgb
# from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
warnings.filterwarnings('ignore')
# from xgboost import XGBClassifier




# Load Data

In [65]:
# train=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Train_FeatureEngineering_4_without_droping_any_feature.csv")
# test=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Test_FeatureEngineering_4.csv")
# for other account
train=pd.read_csv("features/7_Train_FeatureEngineering_didnt_fill_null.csv")
test=pd.read_csv("features/7_Test_FeatureEngineering_didnt_fill_null.csv")

print("Shape of Training Data",train.shape)
print("Shape of Testing Data", test.shape)


Shape of Training Data (17176, 25)
Shape of Testing Data (8576, 24)


In [66]:
train.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'label', 'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'is_outlier_loc',
       'trip_distance'],
      dtype='object')

In [67]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'is_outlier_loc',
       'trip_distance'],
      dtype='object')

# Data_Preprocessing

In [68]:
# drop_columns_train=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
# drop_columns_test=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3','is_outlier_loc']

drop_columns_train=['tripid','pickup_time','drop_time','pickup_date','pickup_day','pickup_month','pickup_year']
drop_columns_test=['tripid','pickup_time','drop_time','pickup_date','pickup_day','pickup_month','pickup_year']

In [69]:
train_1=train.drop(drop_columns_train,axis=1)
test_1=test.drop(drop_columns_test,axis=1)
print("Shape of Training Data after dropping columns",train_1.shape)
print("Shape of Testing Data after dropping columns",test_1.shape)

Shape of Training Data after dropping columns (17176, 18)
Shape of Testing Data after dropping columns (8576, 17)


In [70]:
# def encodeDays(day_of_week):
#   day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
#   return day_dict[day_of_week]
# def encodeLabel(label):
#   label_dict={'correct':1,'incorrect':0}
#   return label_dict[label]
# def clean_data(data,isTrain):
#   data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
#   if(isTrain):
#     data['label']=data['label'].apply(lambda x:encodeLabel(x))
#   return data

# train_1=clean_data(train_1,True)
# test_1=clean_data(test_1,False)
# print("Shape of Training Data after cleaning ",train_1.shape)
# print("Shape of Testing Data after cleaning", test_1.shape)

In [71]:
test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                8576 non-null   float64
 1   duration                       8576 non-null   int64  
 2   meter_waiting                  8576 non-null   int64  
 3   meter_waiting_fare             8576 non-null   float64
 4   meter_waiting_till_pickup      8576 non-null   int64  
 5   pick_lat                       8576 non-null   float64
 6   pick_lon                       8576 non-null   float64
 7   drop_lat                       8576 non-null   float64
 8   drop_lon                       8576 non-null   float64
 9   fare                           8576 non-null   float64
 10  additional_fare_is_outlier     8576 non-null   int64  
 11  meter_waiting_fare_is_outlier  8576 non-null   int64  
 12  fare_is_outlier                8576 non-null   i

In [72]:
columns_titles = ['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','pick_lat','pick_lon','drop_lat','drop_lon','fare','additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_hour','pickup_day_of_week','is_outlier_loc','trip_distance','label']
#columns_titles = ['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','pick_lat','pick_lon','drop_lat','drop_lon','fare','additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_hour','pickup_day_of_week','is_outlier_loc','trip_distance','label','additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier']
train_1=train_1.reindex(columns=columns_titles)
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                16974 non-null  float64
 1   duration                       16974 non-null  float64
 2   meter_waiting                  16974 non-null  float64
 3   meter_waiting_fare             16974 non-null  float64
 4   meter_waiting_till_pickup      16974 non-null  float64
 5   pick_lat                       17176 non-null  float64
 6   pick_lon                       17176 non-null  float64
 7   drop_lat                       17176 non-null  float64
 8   drop_lon                       17176 non-null  float64
 9   fare                           17039 non-null  float64
 10  additional_fare_is_outlier     17176 non-null  int64  
 11  meter_waiting_fare_is_outlier  17176 non-null  int64  
 12  fare_is_outlier                17176 non-null 

In [73]:
categorical_var = ['additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_hour','pickup_day_of_week','is_outlier_loc'] 
# categorical_var = ['pickup_hour'] 
categorical_var_indices = [train_1.columns.get_loc(c) for c in categorical_var if c in train_1]
print('\nCategorical Variables indices : ',categorical_var_indices)


Categorical Variables indices :  [10, 11, 12, 13, 14, 15]


In [74]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                16974 non-null  float64
 1   duration                       16974 non-null  float64
 2   meter_waiting                  16974 non-null  float64
 3   meter_waiting_fare             16974 non-null  float64
 4   meter_waiting_till_pickup      16974 non-null  float64
 5   pick_lat                       17176 non-null  float64
 6   pick_lon                       17176 non-null  float64
 7   drop_lat                       17176 non-null  float64
 8   drop_lon                       17176 non-null  float64
 9   fare                           17039 non-null  float64
 10  additional_fare_is_outlier     17176 non-null  int64  
 11  meter_waiting_fare_is_outlier  17176 non-null  int64  
 12  fare_is_outlier                17176 non-null 

In [75]:
# train_1=pd.get_dummies(train_1)
# test_1=pd.get_dummies(test_1)

# print("Shape of Training Data after One Hot Encoding",train_1.shape)
# print("Shape of Testing Data after One Hot Encoding",test_1.shape)

In [76]:
# print('Old size: %d' % len(train))
# train_1 = train_1.dropna()
# print('New size: %d' % len(train))

In [77]:
X=train_1.drop(['label'],axis=1)
y=train_1['label']

In [78]:
#split data into train and validation data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])

Number of records in training data  13740
Number of records in validation data  3436


# Classifiers

## XGBoost CLassifier

In [79]:
# model = XGBClassifier()
# model.fit(X_train, y_train)

## Linear Regression Classifier

In [80]:
# model = LinearRegression()
# model.fit(X_train,y_train)
# # y_pred=model.predict(X_test)
# # lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
# # print("RMSE for Linear Regression is ",lm_rmse)


## LGBM Classifier

In [81]:
# estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
#                         n_estimators = 20, num_leaves = 38)


# param_grid = {
#     'n_estimators': [x for x in range(24,40,2)],
#     'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
# gridsearch = GridSearchCV(estimator, param_grid)

# gridsearch.fit(X_train, y_train,
#         eval_set = [(X_test, y_test)],
#         eval_metric = ['auc', 'binary_logloss'],
#         early_stopping_rounds = 5)

In [82]:
# gbm = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None,
#                                       colsample_bytree=1.0,
#                                       importance_type='split',
#                                       learning_rate=0.125, max_depth=-1,
#                                       metric='l1', min_child_samples=20,
#                                       min_child_weight=0.001,
#                                       min_split_gain=0.0, n_estimators=20,
#                                       n_jobs=-1, num_leaves=38, objective=None,
#                                       random_state=None, reg_alpha=0.0,
#                                       reg_lambda=0.0, silent=True,
#                                       subsample=1.0, subsample_for_bin=200000,
#                                       subsample_freq=0)


# gbm.fit(X, y,
#         eval_set=[(X_test, y_test)],
#         eval_metric=['auc', 'binary_logloss'],
# early_stopping_rounds=5)

In [83]:
# ax = lgb.plot_importance(gbm, height = 0.4, 
#                          max_num_features = 50, 
#                          xlim = (0,100), ylim = (0,23), 
#                          figsize = (10,6))
# plt.show()

In [84]:
# sorted(gbm.feature_importances_,reverse=True)

In [85]:

# # The code below aims to drop  to keep the features that are included in the most important features. 
# temp = 0 
# total = sum(gbm.feature_importances_)
# for feature in sorted(gbm.feature_importances_, reverse=True):
#     temp+=feature
#     print(feature)
#     if temp/total >= 0.85:
#         print(feature,temp/total) # stop when we 
#         break

In [86]:
# #The above means let go of all variables after PAY_AMT_5
# y_pred_prob = gbm.predict_proba(test_1)[:, 1]
# # auc_roc_0 = str(roc_auc_score(y_test, y_pred_prob)) # store AUC score without dimensionality reduction
# # print('AUC without dimensionality reduction: \n' + auc_roc_0)

In [87]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,additional_fare_is_outlier,meter_waiting_fare_is_outlier,fare_is_outlier,pickup_hour,pickup_day_of_week,is_outlier_loc,trip_distance
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,0,0,0,0,5,0,3.164501
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,0,0,0,0,5,0,1.968540
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,0,0,0,1,5,0,3.917991
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,0,0,0,2,5,0,0.535588
4,NaN,NaN,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,358.39,1,1,0,3,5,0,5.062797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,10.5,838.0,93.0,5.4219,451.0,7.29073,80.6367,7.28891,80.6557,198.26,0,0,0,22,5,0,1.308220
17172,10.5,2151.0,428.0,0.0000,39.0,6.90569,79.8516,6.95089,79.9389,581.23,0,0,0,23,5,0,6.753297
17173,10.5,263.0,9.0,0.0000,110.0,7.09210,79.9000,7.10135,79.9017,76.20,0,0,0,23,5,0,0.649655
17174,10.5,858.0,115.0,0.0000,317.0,6.94540,79.8768,6.93574,79.9010,133.31,0,0,0,23,5,0,1.788975


In [88]:
# #We can choose to drop the last 6 features from in our new model to reduce dimensionality, and thus save training time and space

# X = X_train.drop(['additional_fare','pickup_day_of_week','pickup_month','pickup_day'], axis=1)

## CatBoost

In [89]:
# !pip install catboost

In [90]:
from catboost import CatBoostClassifier


model = CatBoostClassifier(iterations=10000, 
                           cat_features=categorical_var_indices, 
#                            task_type="GPU",
                           devices='0:1',
                           nan_mode='Max')

# model.fit(X,
#           y,
#           verbose=True)




In [ ]:
parameters = {'depth'         : sp_randInt(4, 10),
              'learning_rate' : sp_randFloat(),
              'iterations'    : sp_randInt(10, 100)
             }

randm = RandomizedSearchCV(estimator=model, param_distributions = parameters, 
                           cv = 2, n_iter = 10000, n_jobs=-1)
randm.fit(X, y)

# Results from Random Search
print("\n========================================================")
print(" Results from Random Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      randm.best_estimator_)

print("\n The best score across ALL searched params:\n",
      randm.best_score_)

print("\n The best parameters across ALL searched params:\n",
      randm.best_params_)

print("\n ========================================================")

#Test Model

In [100]:
predictions = model.predict(X_test)
predictions = [round(value) for value in predictions]
np.unique(predictions , return_counts=True) 

(array([0, 1]), array([ 280, 3156]))

In [106]:
accuracy = accuracy_score(y_test, predictions)
f1_macro = f1_score(y_test, predictions, average='macro')
f1_binary = f1_score(y_test, predictions, average='binary')
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Macro F1: %.2f%%" % (f1_macro * 100.0))
print("Binary F1: %.2f%%" % (f1_binary * 100.0))

Accuracy: 97.70%
Macro F1: 93.03%
Binary F1: 98.74%


## Results

Accuracy: 97.26%
Macro F1: 92.10%
Binary F1: 98.49%

### catboost with categorical features removed month,year,nanmode=max 1000 epochs 
Accuracy: 96.07%
Macro F1: 87.50%
Binary F1: 97.85%

### catboost with categorical features removed month,year1000 epochs 
Accuracy: 96.42%
Macro F1: 88.33%
Binary F1: 98.05%

### catboost with categorical features 100000 epochs 
Accuracy: 96.16%
Macro F1: 86.48%
Binary F1: 97.92%
Submission_score : 0.97375

### catboost without dropping any feature with 400000 epochs

Accuracy: 97.41%
Macro F1: 90.87%
Binary F1: 98.60%
Submission Score : 0.97817

### catboost without dropping any feature  with 100000 epochs

<!-- Accuracy: 97.35%
Macro F1: 90.63%
Binary F1: 98.56%
Submission Score : 0.97848 -->

### catboost after normalizing to z score with 10000 epochs

Accuracy: 96.70%
Macro F1: 87.82%
Binary F1: 98.22%

### catboost without dropping any feature with 10000 epochs

Accuracy: 96.82%
Macro F1: 88.95%
Binary F1: 98.27%
Submission Score : 0.97788

# Save Predictions

In [102]:
X_test.info()
test_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3436 entries, 13188 to 12881
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                3396 non-null   float64
 1   duration                       3396 non-null   float64
 2   meter_waiting                  3396 non-null   float64
 3   meter_waiting_fare             3396 non-null   float64
 4   meter_waiting_till_pickup      3396 non-null   float64
 5   pick_lat                       3436 non-null   float64
 6   pick_lon                       3436 non-null   float64
 7   drop_lat                       3436 non-null   float64
 8   drop_lon                       3436 non-null   float64
 9   fare                           3411 non-null   float64
 10  additional_fare_is_outlier     3436 non-null   int64  
 11  meter_waiting_fare_is_outlier  3436 non-null   int64  
 12  fare_is_outlier                3436 non-nul

In [103]:
predictions_1 = model.predict(test_1)
# predictions_1 = [round(value) for value in y_pred_1]
np.unique(predictions_1 , return_counts=True)

(array([0, 1]), array([ 364, 8212]))

In [104]:
submission_df = pd.read_csv("predications_submission.csv", 
                            index_col="tripid")
submission_df['prediction'] = predictions_1

submission_df.to_csv('predictions/3_catboost_categorical(only_day_and_hour)_nanmode_max_100000_epochs_2.csv')

In [105]:
#  y_pred = pd.DataFrame({'prediction':predictions_1})
#  y_pred["prediction"] = y_pred["prediction"].astype(int)
#  y_pred['prediction'] = y_pred['prediction'].apply(lambda x: 0 if x < 0.5 else 1)
#  y_pred.to_csv("predictions/predications_catboost_6_features_3_100000epochs.csv")
# y_pred.to_csv("/content/drive/My Drive/UNI/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/predictions/predications_catboost_6_features_3_400000epochs.csv")